# A3: A\*, IDS, and Effective Branching Factor

Prashant Kumar Thakur

For this assignment, I have implemented the Recursive Best-First Search
implementation of the A\* algorithm given in class with minor modification of code provided in the class notebook.  The functions are named as per the requirement of the assignment and kept in original form.  An additional function `ebf` has been defined that returns an estimate of the effective branching factor for a search algorithm applied to a search problem.

So, the required functions are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search.

The function `iterativeDeepeningSearch` and `aStarSearch` were applied to several eight-tile sliding puzzle
problems. The following functions were also taken from the professors solution code for Assignment 2 with minor modification to handle the requirement of A\* function:

  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step,
  
plus the following function for the eight-tile puzzle to check if the current state equals the goal state:

  * `goalTestF_8p(state, goal)`
  
The following function has been implemented as per the requirement and the results are discussed inline.

   - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])
   


## Heuristic Functions

For `aStarSearch` use the following two heuristic functions, plus one more of my own design, for a total of three heuristic functions.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = ?$, Similar to h2_8p but instead of returning the Manhattan distance of only blank space, total number of misplaced tiles are considered to compute the aggregate Manhattan distance.

## Comparison

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

This representation has been discussed below the code description.

# Function Declaration

In [1]:
from math import sqrt, ceil
class Node:
    def __init__(self, state, f=0, g=0,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"


#############################################################################################
# IDS and DS
def iterativeDeepeningSearch(startState, goalState,actionsF, takeActionF, maxDepth):
    global nNode
    nNode = [0,0]  # [depth,nodes]
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState,actionsF, takeActionF, depth)
        if result is 'failure':
            return 'failure'
        if result is not 'cutoff':
            result.insert(0, startState)
            return (result)
    return 'cutoff'


def depthLimitedSearch(state, goalState,actionsF, takeActionF, depthLimit):
#     global nNode
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        # Find all expanded nodes
        nNode[1] +=1
        childState,_= takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState,actionsF, takeActionF, depthLimit-1)
        if result is 'cutoff':
            cutoffOccurred = True
        elif result is not 'failure':
            # Find total depth
            nNode[0]+=1
            result.insert(0, childState)
            return result
    if cutoffOccurred:
        return 'cutoff'
    else:
        return 'failure'


# End of IDS and DS
#####################################################################################
# A-start start
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    global aNode
    aNode = [0,0] # [depth,nodes]
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))


def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    if goalTestF(parentNode.state):
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return ("failure", float('inf'))
    children = []
    for action in actions:
        # Find all expanded nodes
        aNode[1] += 1
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax or bestChild.f == float('inf'):
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                            hF, min(fmax,alternativef))
        if result is not "failure":
            # Gives depth of search
            aNode[0] += 1
            result.insert(0,parentNode.state)
            return (result, bestChild.f)

#End of A-Start
######################################################################
# Eight Puzzle


def findBlank_8p(s):
    return iTorc(s.index(0))


def actionsF_8p(state):
    r, c = findBlank_8p(state)
    actions = []
    if c > 0:
        actions.append(('left',1))
    if c < 2:
        actions.append(('right',1))
    if r > 0:
        actions.append(('up',1))
    if r < 2:
        actions.append(('down',1))
    return actions


def takeActionF_8p(state, action):
    import copy
    # action passed has (actual_action, cost)
    action,cost = action
    state = copy.deepcopy(state)
    r, c = findBlank_8p(state)
    dr = dc = 0
    if action is 'left':
        dc -= 1
    elif action is 'right':
        dc += 1
    elif action is 'up':
        dr -= 1
    elif action is 'down':
        dr += 1
    newr, newc = r+dr, c+dc
    setTile(state, r, c, getTile(state, newr, newc))
    setTile(state, newr, newc, 0)
    return (state,cost)


def goalTestF_8p(s, goalState):
    return s == goalState

def rcToi(row, col):
    return row*3+col


def iTorc(i):
    row = i // 3
    col = i - row*3
    return (row, col)

def setTile(state, row, col, tile):
    state[rcToi(row, col)] = tile
    return state

def getTile(state, row, col):
    return state[rcToi(row, col)]
# End of 8-puzzle

###################################################
# Heuristic, EBF and runExperiment
def h1_8p(state,goal):
    return 0

#Manhattan distance of blank space only
def h2_8p(state,goal):
    sc = findBlank_8p(state)
    gc = findBlank_8p(goal)
    dist = abs(gc[0]-sc[0])+abs(gc[1]-sc[1])
    return dist

#Ecludian distance of blank space
def h4_8p(state,goal):
    sx,sy = findBlank_8p(state)
    gx,gy = findBlank_8p(goal)
    return ceil(sqrt((gx-sx)**2+(gy-sy)**2))


# Manhattan distance - sum of all misplaced values
def h3_8p(state,goal):
    sum = 0
    for item in state:
        sc = iTorc(state.index(item))
        gc = iTorc(goal.index(item))
        if sc != gc and item != 0:
            sum += abs(sc[0]-gc[0])+abs(sc[1]-gc[1])
    return sum

def eqn_sum(c,d):
    sum = 1
    while d != 0:
        sum += c**(d)
        d = d-1
    return sum

def ebf(N,d,precision=0.01):
    if N <=0:
        return 0.0
    low = 0.0
    high = N
    mid = float(high)
    while abs(eqn_sum(mid,d)-N) > precision:
        mid = (low+high)/2.0
        if eqn_sum(mid,d) > N:
            high = mid
        else:
            low = mid
    return mid

def runExperiment(goalState1, goalState2, goalState3,h):
    # Start state of each function
    state = [1,2,3,4,0,5,6,7,8]
    # MAX DEPTH OF IDS
    max_depth = 15
    print('\t{}\t{}\t{}\n'.format(goalState1,goalState2,goalState3))
    print('{}{:>10}{:>7}{:>8}{:>15}{:>10}{:>8}{:>15}{:>10}{:>8}'.format('Algorithm','Depth','Nodes','EBF','Depth','Nodes','EBF','Depth','Nodes','EBF'))
    value_ids=["IDS"]
    value_as ={'h1_8p':["A*h1"], 'h2_8p':["A*h2"],'h3_8p':["A*h3"]}
    for i in [goalState1,goalState2,goalState3]:
        iterativeDeepeningSearch(state, i, actionsF_8p, takeActionF_8p, maxDepth=max_depth)
        value_ids.extend(nNode)
        value_ids.append(ebf(nNode[1],nNode[0]))
        for hf in h:
            aStarSearch(state, actionsF_8p, takeActionF_8p, lambda s: goalTestF_8p(s, i),lambda s: hf(s, i))
            value_as[hf.__name__].extend(aNode)
            value_as[hf.__name__].append(ebf(aNode[1],aNode[0]))
    print('{}\t\t{}\t{}\t{:.3f}\t\t{}\t{}\t{:.3f}\t\t{}\t{}\t{:.3f}'.format(value_ids[0],value_ids[1],value_ids[2],value_ids[3],value_ids[4],value_ids[5],value_ids[6],value_ids[7],value_ids[8],value_ids[9]))
    print('{}\t\t{}\t{}\t{:.3f}\t\t{}\t{}\t{:.3f}\t\t{}\t{}\t{:.3f}'.format(value_as['h1_8p'][0],value_as['h1_8p'][1],value_as['h1_8p'][2],value_as['h1_8p'][3],value_as['h1_8p'][4],value_as['h1_8p'][5],value_as['h1_8p'][6],value_as['h1_8p'][7],value_as['h1_8p'][8],value_as['h1_8p'][9]))
    print('{}\t\t{}\t{}\t{:.3f}\t\t{}\t{}\t{:.3f}\t\t{}\t{}\t{:.3f}'.format(value_as['h2_8p'][0],value_as['h2_8p'][1],value_as['h2_8p'][2],value_as['h2_8p'][3],value_as['h2_8p'][4],value_as['h2_8p'][5],value_as['h2_8p'][6],value_as['h2_8p'][7],value_as['h2_8p'][8],value_as['h2_8p'][9]))
    print('{}\t\t{}\t{}\t{:.3f}\t\t{}\t{}\t{:.3f}\t\t{}\t{}\t{:.3f}'.format(value_as['h3_8p'][0],value_as['h3_8p'][1],value_as['h3_8p'][2],value_as['h3_8p'][3],value_as['h3_8p'][4],value_as['h3_8p'][5],value_as['h3_8p'][6],value_as['h3_8p'][7],value_as['h3_8p'][8],value_as['h3_8p'][9]))


In [2]:
goal1=[1, 2, 3, 4, 0, 5, 6, 7, 8]
goal2=[1, 2, 3, 4, 5, 8, 6, 0, 7]
goal3=[1, 0, 3, 4, 5, 8, 2, 6, 7]
runExperiment(goal1,goal2,goal3,[h1_8p,h2_8p,h3_8p])

	[1, 2, 3, 4, 0, 5, 6, 7, 8]	[1, 2, 3, 4, 5, 8, 6, 0, 7]	[1, 0, 3, 4, 5, 8, 2, 6, 7]

Algorithm     Depth  Nodes     EBF          Depth     Nodes     EBF          Depth     Nodes     EBF
IDS		0	0	0.000		3	43	3.086		11	225850	2.954
A*h1		0	0	0.000		3	116	4.488		11	643246	3.263
A*h2		0	0	0.000		3	51	3.297		11	100046	2.733
A*h3		0	0	0.000		3	9	1.578		11	1172	1.762


### Description of above table.

The above function is the implementation of `runExperiment` function which takes three goalStates as input along with the list of three heuristic functions. Internally, the code runs the `iterativeDeependingSearch` and nodes the depth, nodes returned by the function and compute its effective branching factor. Similarly `aStartSearch` is also called for three different heuristics functions to get the corresponding number of nodes, depth of search and the effective branching factor. We can see if the `goalState` and `startState` are similar, all the functions returned 0 for nodes, depth and effective branching factor. However if we pass the second `goalState` to all of them, we see there is considerable improvement in `aStartSearch` with better heuristic function. The IDS explores 43 nodes to depth of 3 with an effective branching factor of 3.086 while the `aStartSearch` sturggled with h1_8p (which is a poor heuristics that returns 0) to get to the final result with 116 explored nodes and 4.488 effective branching factor. Additionally, when the heuristics function gets better in predicting the cost to reach the goal the goal was achieved quickly with less effective branching factor. We can see from the table above that for the third heuristic function, the `aStarSearch` gets to the goal by expanding only 9 nodes with an effective branching factor of 1.578. Similarly, the result is consistent with the third goal state. This explains that if we have better heuristic function which is admissible then the effective branching factor is reduced which means the number of nodes that need to be expanded at each depth decreases and the goal state can be reached faster.  

### Is h3_8p admissible?

Yes, the third heuristic function is admissible as it never overestimates the cost to reach the goal state. Let's take few examples to demonstrate the cost to reach the goal estimated by the thrid heuristics.

    startState = [1, 2, 3, 4, 0, 5, 6, 7, 8]
    goalState1 = [1, 2, 3, 4, 0, 5, 6, 7, 8]
    goalState2 = [1, 2, 3, 4, 5, 8, 6, 0, 7]
    goalState3 = [1, 0, 3, 4, 5, 8, 2, 6, 7]
    
If we take goalState1 and pass it to the h3 to find the number of steps that would lead to the goal state, the h3 computes the Manhattan distance of the misplaced tiles. Since, there are no misplaced tiles the h3 returns 0 as its output and which is exactly the number of state to reach the goal. This is a perfect estimation as seen below.
Similarly, if we consider the goalState2 we see the estimated steps to reach the goal is 4 which is the sum of all the steps required to swipe the tiles to the goal position. 

    Sum = 1 (steps to swipe 5 to its goal position) + 1 ( Steps to swipe 8 to its goal position) + 1 (steps to swipe 7 to its goal position) =3

If we try to get the number of steps to reach the goal we can see the shortest path to reach the goal is:
    
    Solution Path to [1, 2, 3, 4, 5, 8, 6, 0, 7]:  [[1, 2, 3, 4, 0, 5, 6, 7, 8], [1, 2, 3, 4, 5, 0, 6, 7, 8], [1, 2, 3, 4, 5, 8, 6, 7, 0], [1, 2, 3, 4, 5, 8, 6, 0, 7]]. 
    
As we can see in the result the number of transition required from the startState to reach the goalState1 is 3 which was what estimated by the heuristic function. We are luck to get the estimate accurately again. 
Now let's consider the third goal and we see the h3_8p estimates 7 steps to reach the goal. Now if we try to expand the result and see what are the steps to reach the goal through the startState we see there are 11 steps required to reach the goal. The function does not overestimates in this case as well.
    
    Solution path to goalState3: [[1, 2, 3, 4, 0, 5, 6, 7, 8], [1, 0, 3, 4, 2, 5, 6, 7, 8], [0, 1, 3, 4, 2, 5, 6, 7, 8], [4, 1, 3, 0, 2, 5, 6, 7, 8], [4, 1, 3, 2, 0, 5, 6, 7, 8], [4, 1, 3, 2, 5, 0, 6, 7, 8], [4, 1, 3, 2, 5, 8, 6, 7, 0], [4, 1, 3, 2, 5, 8, 6, 0, 7], [4, 1, 3, 2, 5, 8, 0, 6, 7], [4, 1, 3, 0, 5, 8, 2, 6, 7], [0, 1, 3, 4, 5, 8, 2, 6, 7], [1, 0, 3, 4, 5, 8, 2, 6, 7]]
    
Lets check for the estimate for the second state found in the solution path [1, 0, 3, 4, 2, 5, 6, 7, 8] we get the estimate of 6 which is again lower than the total cost required to reach the goal i.e. 10. Finally, if we check the heuristic to estimate the cost to reach the goal via  [4, 1, 3, 0, 5, 8, 2, 6, 7], we find the estimate to be 2 (as shown below) which is again an exact cost to reach the goal. 
    
Hence, there are no state to which this function over-estimates. Since the heuristic to be admissible, it has been defined to never over-estimate.
    
    

In [55]:
startState = [1, 2, 3, 4, 0, 5, 6, 7, 8]
goalState1 = [1, 2, 3, 4, 0, 5, 6, 7, 8]
h3_8p(startState,goalState1)

0

In [56]:
goalState2 = [1, 2, 3, 4, 5, 8, 6, 0, 7]
h3_8p(startState,goalState2)

3

In [57]:
goalState3 = [1, 0, 3, 4, 5, 8, 2, 6, 7]
h3_8p(startState,goalState3)

7

In [58]:
startState=[1, 0, 3, 4, 2, 5, 6, 7, 8]
h3_8p(startState,goalState3)

6

In [59]:
startState=[4, 1, 3, 0, 5, 8, 2, 6, 7]
h3_8p(startState,goalState3)

2

First, some example output for the ebf function.

In [60]:
ebf(10, 3)

1.66015625

The smallest argument values should be a depth of 0, and 1 node.

In [61]:
ebf(1, 0)

1.0

In [62]:
ebf(2, 1)

1.0

Note: The above examples shows that the effective branching factor with 2 nodes and 1 depth is 1.0. The algorithm finds the solution for this problem exactly with no error at all. 

In [63]:
ebf(2, 1, precision=0.000001)

1.0

In [64]:
ebf(200000, 5)

11.275596989435144

In [65]:
ebf(200000, 50)

1.234819248452368

Here is a simple example using the usual simple graph search.

In [66]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['e', 'f', 'g'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [67]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [68]:
takeActionF_simple('a', actions[0])

('b', 1)

In [69]:
goalTestF_simple('a', 'a')

True

In [70]:
h_simple('a', 'z')

1

In [71]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [72]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [73]:
%run -i A3grader.py


Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8], [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)

Testing iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], 
                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],
                            